#### This notebook looks at temperature-dependent changes to embryo morphology

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import os
from glob2 import glob
from src.functions.plot_functions import format_3d_plotly, rotate_figure, format_2d_plotly

In [ ]:
# load embryo_df for our current best model
# root = "/media/nick/hdd02/Cole Trapnell's Lab Dropbox/Nick Lammers/Nick/morphseq/"

root = "/Users/nick/Cole Trapnell's Lab Dropbox/Nick Lammers/Nick/morphseq/"

# path to save data
read_path = os.path.join(root, "results", "20250312", "morph_latent_space", "")

# path to figures and data
fig_path = "/Users/nick/Cole Trapnell's Lab Dropbox/Nick Lammers/Nick/slides/morphseq/20250513/morph_metrics/"
fig_data_path = "/Users/nick/Cole Trapnell's Lab Dropbox/Nick Lammers/Nick/slides/morphseq/20250513/data/morph_metrics/"
os.makedirs(fig_path, exist_ok=True)
os.makedirs(fig_data_path, exist_ok=True)

In [ ]:
import joblib

# load datasets
hf_pca_df = pd.read_csv(os.path.join(read_path, "hf_pca_morph_df.csv"))
ref_pca_df = pd.read_csv(os.path.join(read_path, "ab_ref_pca_morph_df.csv"))
spline_df = pd.read_csv(os.path.join(read_path, "spline_morph_df_full.csv"))
spline_df["knot_index"] = spline_df.index

# Save the model to a file
morph_stage_model = joblib.load(os.path.join(read_path, 'morph_stage_model.joblib'))

## Plot ref trajectories

In [ ]:
from tqdm import tqdm 

angle_vec = np.linspace(0, 360, 50)
t_lim_vec = np.linspace(12, 48, 50)
marker_size = 4

frame_path = os.path.join(fig_path, "ref_pca_rot_frames", "")
os.makedirs(frame_path, exist_ok=True)


# set plot parameters
zoom_factor = 0.21
z_rotation = -30 + 102
elevation = -10
marker_size = 3

xrange = [-2.2, 2.5]
yrange = [-2.7, 1.6]
zrange = [-2.1, 1.7]

pca_axis_labels=["morph PC 1", "morph PC 2", "morph PC 3"]
# make fig


# fig.show()
# for a, angle in enumerate(tqdm(angle_vec)):
    
    
#     fig = rotate_figure(fig, zoom_factor=zoom_factor, z_rotation=z_rotation + angle, elev_rotation=elevation)
    

#     fig.write_image(os.path.join(frame_path, f"hotfish_pca_rot{a:02}.png"), scale=2)
#     # fig.write_html(os.path.join(fig_path, f"hotfish_pca_rot{a:02}.html"))
    
# fig.show()
# fig.write_image(os.path.join(fig_path, "ref_pca_all.png"), scale=2)
# fig.write_html(os.path.join(fig_path, "ref_pca_all.html"))


# surf_frame_path = os.path.join(fig_path, "hf_morph_surf_frames", "")
# os.makedirs(surf_frame_path, exist_ok=True)

for t, t_lim in enumerate(tqdm(t_lim_vec)):
    angle = angle_vec[t]
    
    t_filter = ref_pca_df["mdl_stage_hpf"] <= t_lim 
    # if np.sum(t_filter) == 0:
    #     opacity = 0
    #     t_filter = ref_pca_df["mdl_stage_hpf"] <= np.inf
    # else:
    #     opacity = 1
        # t_filter = hf_pca_df["mdl_stage_hpf"] <= t_lim 

    fig = px.scatter_3d(ref_pca_df.loc[t_filter], x="PCA_00_bio", y="PCA_01_bio", z="PCA_02_bio", 
                    color="mdl_stage_hpf", opacity=1, range_color=[10, 42],
                    hover_data={"snip_id"})

    fig.layout.scene.xaxis.range = xrange
    fig.layout.scene.yaxis.range = yrange
    fig.layout.scene.zaxis.range = zrange

    
    fig = format_3d_plotly(fig, axis_labels=pca_axis_labels, aspectmode="manual",theme="light", 
                           marker_size=marker_size, marker_edge=True)

    grid_dict = dict(
          tickmode="array",     # custom list
          tickvals=[-2, -1, 0, 1, 2],
          ticktext=["-2","-1","0","1","2"],
          showgrid=True)
    
    fig.update_layout(
      scene=dict(
        xaxis=grid_dict,
        yaxis=grid_dict,
        zaxis=grid_dict
      ))
    
    
    fig = rotate_figure(fig, zoom_factor=zoom_factor, z_rotation=z_rotation + angle, elev_rotation=elevation)

    fig.update_layout(
          # coloraxis_showscale=False,                       # hide
          # or:
          coloraxis_showscale=True,
          coloraxis_colorbar=dict(title="embryo stage (hpf)")     # relabel
        )
    
    fig.write_image(os.path.join(frame_path, f"ref_pca_ab_angle{t:02}.png"), scale=2)

fig.show()

### Now add lines for individual embryo

In [ ]:
start_stage = 12
stop_stage = 38
# get list of embryo IDs
embryo_df = ref_pca_df.loc[:, [ "embryo_id", "mdl_stage_hpf"]].groupby(
                        [ "embryo_id"])["mdl_stage_hpf"].agg(["min", "max"]).reset_index()

stage_filter = (embryo_df["min"] <= start_stage) & (embryo_df["max"] >= stop_stage)

# indices_to_plot = np.asarray([1, 4, 7])
# embryo_id_index = np.unique(ref_pca_df["embryo_id"])
embryo_ids_to_plot = embryo_df.loc[stage_filter, "embryo_id"].to_numpy()
n_plot = 3
# np.sum(stage_filter)

In [ ]:
fig = px.scatter_3d(ref_pca_df, x="PCA_00_bio", y="PCA_01_bio", z="PCA_02_bio", 
                    color="mdl_stage_hpf", opacity=.1, range_color=[10, 42],
                    hover_data={"snip_id"})

fig.layout.scene.xaxis.range = xrange
fig.layout.scene.yaxis.range = yrange
fig.layout.scene.zaxis.range = zrange

fig = format_3d_plotly(fig, axis_labels=pca_axis_labels, aspectmode="manual",theme="light", marker_size=3, marker_edge=False)

grid_dict = dict(
          tickmode="array",     # custom list
          tickvals=[-2, -1, 0, 1, 2],
          ticktext=["-2","-1","0","1","2"],
          showgrid=True)
    
fig.update_layout(
      scene=dict(
        xaxis=grid_dict,
        yaxis=grid_dict,
        zaxis=grid_dict
      ))
    
fig = rotate_figure(fig, zoom_factor=zoom_factor, z_rotation=z_rotation, elev_rotation=elevation)

fig.update_layout(
          # coloraxis_showscale=False,                       # hide
          # or:
          coloraxis_showscale=True,
          coloraxis_colorbar=dict(title="embryo stage (hpf)")     # relabel
        )

fig.write_image(os.path.join(fig_path, f"embryo_lines{0:02}.png"), scale=2)
for e, embryo_id in enumerate(embryo_ids_to_plot[:n_plot]):
    plot_filter = ref_pca_df["embryo_id"] == embryo_id
    fig.add_traces(go.Scatter3d(x=ref_pca_df.loc[plot_filter, "PCA_00_bio"], 
                     y=ref_pca_df.loc[plot_filter, "PCA_01_bio"], 
                     z=ref_pca_df.loc[plot_filter, "PCA_02_bio"],
                     mode="markers+lines", marker=dict(size=5), line=dict(width=2),
                               showlegend=False))
    
    fig.write_image(os.path.join(fig_path, f"embryo_lines{e+1:02}.png"), scale=2)

fig.show()

### Flux plot

In [ ]:
from tqdm import tqdm

pca_cols = [col for col in ref_pca_df.columns if "PCA" in col]

# calculate per-point velocity for each embryo
embryo_index = np.unique(ref_pca_df["embryo_id"])

ref_pca_df.loc[:, "PCA_00_3_vel"] = np.nan
ref_pca_df.loc[:, "PCA_01_3_vel"] = np.nan
ref_pca_df.loc[:, "PCA_02_3_vel"] = np.nan

for e, embryo in enumerate(tqdm(embryo_index, "Extracting embryo velocities...")):
    s_indices = np.where(ref_pca_df["embryo_id"]==embryo)[0]
    
    u0 = ref_pca_df.loc[s_indices, pca_cols[0]].to_numpy()
    u1 = ref_pca_df.loc[s_indices, pca_cols[1]].to_numpy()
    u2 = ref_pca_df.loc[s_indices, pca_cols[2]].to_numpy()

    if len(u0) > 5:
    
        t = ref_pca_df.loc[s_indices, "mdl_stage_hpf"].to_numpy()
        
        du0 = np.divide(np.diff(u0), np.diff(t))
        du1 = np.divide(np.diff(u1), np.diff(t))
        du2 = np.divide(np.diff(u2), np.diff(t))
        
        ref_pca_df.loc[s_indices[:-1], "PCA_00_3_vel"] = du0
        ref_pca_df.loc[s_indices[:-1], "PCA_01_3_vel"] = du1
        ref_pca_df.loc[s_indices[:-1], "PCA_02_3_vel"] = du2
    
        ref_pca_df.loc[s_indices[-1], "PCA_00_3_vel"] = du0[-1]
        ref_pca_df.loc[s_indices[-1], "PCA_01_3_vel"] = du1[-1]
        ref_pca_df.loc[s_indices[-1], "PCA_02_3_vel"] = du2[-1]

In [ ]:
from sklearn.cluster import KMeans

cluster_size = 25
max_pert_clusters = 500
min_pert_clusters = 10

# pull out raw PCA coordinates

pca_array = ref_pca_df.loc[:, pca_cols[:3]].to_numpy()
pca_v_array = ref_pca_df.loc[:, ["PCA_00_3_vel", "PCA_01_3_vel", "PCA_02_3_vel"]].to_numpy()
age_vec = ref_pca_df.loc[:, "mdl_stage_hpf"].to_numpy()
n_points = pca_array.shape[0]

n_clusters = np.min([int(np.round(n_points/cluster_size)), max_pert_clusters])

# cluster
kmeans_out = KMeans(n_clusters=n_clusters, random_state=0, n_init="auto").fit(pca_array)
label_vec = kmeans_out.labels_

label_index, ia, label_counts = np.unique(label_vec, return_counts=True, return_inverse=True)

ref_pca_df.loc[:, "kmeans_label"] = label_vec
ref_pca_df.loc[:, "cluster_counts"] = label_counts[ia]

label_index = label_index[label_counts>15]
label_counts = label_counts[label_counts>15]

k_df = pd.DataFrame(label_index, columns=["kmeans_label"])
k_df["cluster_counts"] = label_counts
k_df["n_clusters"] = n_clusters
for l, lb in enumerate(label_index):
    
    k_df.loc[l, "X"] = np.mean(pca_array[label_vec==lb, 0])
    k_df.loc[l, "Y"] = np.mean(pca_array[label_vec==lb, 1])
    k_df.loc[l, "Z"] = np.mean(pca_array[label_vec==lb, 2])

    k_df.loc[l, "dX"] = np.mean(pca_v_array[label_vec==lb, 0])
    k_df.loc[l, "dY"] = np.mean(pca_v_array[label_vec==lb, 1])
    k_df.loc[l, "dZ"] = np.mean(pca_v_array[label_vec==lb, 2])

    k_df.loc[l, "stage_hpf"] = np.mean(age_vec[label_vec==lb])

# cluster_df_list.append(k_df)


cluster_df = k_df

In [ ]:
import plotly.graph_objects as go

dirs = np.stack([cluster_df.dX, cluster_df.dY, cluster_df.dZ], axis=1)
norms = np.linalg.norm(dirs, axis=1, keepdims=True)
unit_dirs = dirs / (norms + 1e-8)

# assume cluster_df is your DataFrame
# you may need to import pandas and read it in first:
# import pandas as pd
# cluster_df = pd.read_csv("…")
offset = 50

fig = go.Figure(
    data=go.Cone(
        x=cluster_df["X"],
        y=cluster_df["Y"],
        z=cluster_df["Z"],
        u         = unit_dirs[:, 0] * cluster_df.stage_hpf + offset*unit_dirs[:, 0],
        v         = unit_dirs[:, 1] * cluster_df.stage_hpf + offset*unit_dirs[:, 1],
        w         = unit_dirs[:, 2] * cluster_df.stage_hpf + offset*unit_dirs[:, 2],
        # colorscale="Viridis",                   # or any other Plotly scale
        cmin=cluster_df["stage_hpf"].min() + offset,
        cmax=cluster_df["stage_hpf"].max() + offset,
        # intensity=cluster_df["stage_hpf"],          # color by your stage
        showscale=True,                         # show colorbar
        sizemode="absolute",                    # each cone has same scale
        sizeref   = (cluster_df.stage_hpf.max() - cluster_df.stage_hpf.min()) * 4
    )
)

fig.update_layout(
    scene=dict(
        xaxis_title="X",
        yaxis_title="Y",
        zaxis_title="Z",
        aspectmode="data"   # ensures x/y/z are equally scaled
    ),
    title="3D Velocity Field"
)

fig.show()

In [ ]:
# 1) your real cones, sized by true speed but with showscale=False
speeds = np.linalg.norm(cluster_df[["dX","dY","dZ"]], axis=1)
cone = go.Cone(
    x=cluster_df.X, y=cluster_df.Y, z=cluster_df.Z,
    u=cluster_df.dX, v=cluster_df.dY, w=cluster_df.dZ,
    colorscale="Greys",
    cmin=speeds.min(), cmax=speeds.max(),
    showscale=False,    # hide the cone’s own colorbar
    sizemode="absolute",
    sizeref=speeds.max(),
    opacity=0.8
)

# 2) an invisible scatter just to get the colorbar for stage_hpf
scatter_color = go.Scatter3d(
    x=cluster_df.X, y=cluster_df.Y, z=cluster_df.Z,
    mode="markers",
    marker=dict(
        size=0,  # invisible
        color=cluster_df.stage_hpf,
        colorscale="Viridis",
        cmin=cluster_df.stage_hpf.min(),
        cmax=cluster_df.stage_hpf.max(),
        colorbar=dict(title="Stage (hpf)"),
        showscale=True
    ),
    showlegend=False
)

fig = go.Figure((cone, scatter_color))
fig.update_layout(
    scene=dict(
        xaxis_title="X", yaxis_title="Y", zaxis_title="Z",
        aspectmode="data"
    ),
    title="3D Velocity Field\n(cones sized by speed, colored by stage_hpf)"
)
fig.show()

In [ ]:
import numpy as np
import plotly.graph_objects as go

# 1) compute a robust length scale
vecs = cluster_df[["dX","dY","dZ"]].values
lengths = np.linalg.norm(vecs, axis=1)
scale = 1.0 / np.percentile(lengths, 90)   # now 90% of arrows ≤ length 1

# 2) build one trace per arrow, scaling dx,dy,dz
arrow_traces = []
for row, L in zip(cluster_df.itertuples(), lengths):
    x0, y0, z0 = row.X, row.Y, row.Z
    v = np.sqrt(row.dX**2 + row.dY**2 + row.dZ**2)
    dx, dy, dz = row.dX * scale / v, row.dY * scale / v, row.dZ * scale / v

    arrow_traces.append(
        go.Scatter3d(
            x=[x0, x0+dx],
            y=[y0, y0+dy],
            z=[z0, z0+dz],
            mode="lines",
            line=dict(
                color=row.stage_hpf,          # color by continuous stage
                colorscale="Viridis",
                cmin=cluster_df.stage_hpf.min(),
                cmax=cluster_df.stage_hpf.max(),
                width=4
            ),
            showlegend=False
        )
    )
    
# 3) a tiny dummy scatter purely to draw the continuous colorbar:
colorbar_trace = go.Scatter3d(
    x=cluster_df.X,
    y=cluster_df.Y,
    z=cluster_df.Z,
    mode="markers",
    marker=dict(
        size=0.1,  # essentially invisible
        color=cluster_df.stage_hpf,
        colorscale="Viridis",
        cmin=cluster_df.stage_hpf.min(),
        cmax=cluster_df.stage_hpf.max(),
        colorbar=dict(title="Stage (hpf)"),
        showscale=True
    ),
    hoverinfo="skip",
    showlegend=False
)

fig = go.Figure(arrow_traces + [colorbar_trace])
fig.update_layout(
    scene=dict(
        xaxis_title="X",
        yaxis_title="Y",
        zaxis_title="Z",
        aspectmode="data"
    ),
    title="3D Quiver (normalized length, colored by continuous stage)"
)
fig.show()

### Make 3D PCA plot for embryos from hotfish experiments

In [ ]:
cluster_df["X"]

### Make series of plots illustrating the location of different temperatures along the trajectory

In [ ]:
embryo_index = np.unique(ref_pca_df["embryo_id"])
n_plot = 75

In [ ]:
from tqdm import tqdm 
z_rotation = 72
elevation = -10


temps_to_plot = np.asarray([28.5, 19, 25, 32, 33.5, 35])
times_to_plot = np.asarray([24, 30, 36])
marker_size = 4

for t in tqdm(range(len(temps_to_plot)+1)):    

    if t == 0:
        opacity=0
        tlim=temps_to_plot
    #     fig = go.Figure()
    else:
        opacity=1
        tlim = np.asarray(temps_to_plot[:t])
    temp_filter = np.isin(hf_pca_df["temperature"], tlim)
    fig = px.scatter_3d(hf_pca_df.loc[temp_filter], x="PCA_00_bio", y="PCA_01_bio", z="PCA_02_bio", 
                        color="temperature", symbol="timepoint", opacity=opacity,
                        color_continuous_scale="RdBu_r", range_color=[19, 38], 
                        hover_data={"timepoint", "snip_id"})
    
    for e, embryo_id in enumerate(embryo_index[:n_plot]):
        plot_filter = ref_pca_df["embryo_id"] == embryo_id
        fig.add_traces(go.Scatter3d(x=ref_pca_df.loc[plot_filter, "PCA_00_bio"], 
                         y=ref_pca_df.loc[plot_filter, "PCA_01_bio"], 
                         z=ref_pca_df.loc[plot_filter, "PCA_02_bio"],
                         mode="lines", line=dict(
                                color=ref_pca_df.loc[plot_filter, "mdl_stage_hpf"], 
                                cmin=10,
                                width=1), showlegend=False, opacity=0.25))
    
    fig.layout.scene.xaxis.range = xrange
    fig.layout.scene.yaxis.range = yrange
    fig.layout.scene.zaxis.range = zrange
    
    pca_axis_labels=["morph PC 1", "morph PC 2", "morph PC 3"]
    fig = format_3d_plotly(fig, axis_labels=pca_axis_labels, theme="light", marker_size=marker_size, font_size=12)
    
    fig.update_layout(
      scene=dict(
        # 1) lock in the ranges you want
        xaxis=dict(range=xrange, autorange=False),
        yaxis=dict(range=yrange, autorange=False),
        zaxis=dict(range=zrange, autorange=False),
        # 2) keep aspect‐ratio constant
        aspectmode="manual",
        aspectratio=dict(
          x=(xrange[1]-xrange[0]),
          y=(yrange[1]-yrange[0]),
          z=(zrange[1]-zrange[0])
        ),
        # 3) fix the camera “eye” (distance + angle) once
        camera=dict(
          eye=dict(x=1.2, y=1.2, z=0.8)
        )
      )
    )

    fig = rotate_figure(fig, zoom_factor=zoom_factor*0.95, z_rotation=z_rotation, elev_rotation=elevation)
    
    grid_dict = dict(
          tickmode="array",     # custom list
          tickvals=[-2, -1, 0, 1, 2],
          ticktext=["-2","-1","0","1","2"],
          showgrid=True)
    
    fig.update_layout(
          scene=dict(
            xaxis=grid_dict,
            yaxis=grid_dict,
            zaxis=grid_dict
          ))
    fig.update_layout(
          coloraxis_showscale=False,  
          showlegend=False# hide
        )
    
    fig.write_image(os.path.join(fig_path, f"hotfish_pca_temp{t:02}.png"), scale=2)
    fig.write_html(os.path.join(fig_path, f"hotfish_pca_temp{t:02}.html"))
    
fig.show()



### Make rotating plot

In [ ]:
start_z_rotation = 72
stop_z_rotation = 360 + 15
start_elevation = -10
stop_elevation = 10
a_vec = np.linspace(start_z_rotation, stop_z_rotation, 50)
e_vec = np.linspace(start_elevation, stop_elevation, 50)

In [ ]:
temp_frame_path = os.path.join(fig_path, "temp_pca_rot_frames", "")
os.makedirs(temp_frame_path, exist_ok=True)
marker_size = 4

for i in tqdm(range(len(e_vec))):
    elevation = e_vec[i]
    z_rotation = a_vec[i]
    
    fig = px.scatter_3d(hf_pca_df.loc[temp_filter], x="PCA_00_bio", y="PCA_01_bio", z="PCA_02_bio", 
                            color="temperature", symbol="timepoint", opacity=opacity,
                            color_continuous_scale="RdBu_r", range_color=[19, 38], 
                            hover_data={"timepoint", "snip_id"})
        
    for e, embryo_id in enumerate(embryo_index[:n_plot]):
        plot_filter = ref_pca_df["embryo_id"] == embryo_id
        fig.add_traces(go.Scatter3d(x=ref_pca_df.loc[plot_filter, "PCA_00_bio"], 
                         y=ref_pca_df.loc[plot_filter, "PCA_01_bio"], 
                         z=ref_pca_df.loc[plot_filter, "PCA_02_bio"],
                         mode="lines", line=dict(
                                color=ref_pca_df.loc[plot_filter, "mdl_stage_hpf"], 
                                cmin=10,
                                width=1), showlegend=False, opacity=0.25))
    
    fig.layout.scene.xaxis.range = xrange
    fig.layout.scene.yaxis.range = yrange
    fig.layout.scene.zaxis.range = zrange
    
    pca_axis_labels=["morph PC 1", "morph PC 2", "morph PC 3"]
    fig = format_3d_plotly(fig, axis_labels=pca_axis_labels, theme="light", marker_size=marker_size, font_size=12)
    
    fig.update_layout(
      scene=dict(
        # 1) lock in the ranges you want
        xaxis=dict(range=xrange, autorange=False),
        yaxis=dict(range=yrange, autorange=False),
        zaxis=dict(range=zrange, autorange=False),
        # 2) keep aspect‐ratio constant
        aspectmode="manual",
        aspectratio=dict(
          x=(xrange[1]-xrange[0]),
          y=(yrange[1]-yrange[0]),
          z=(zrange[1]-zrange[0])
        ),
        # 3) fix the camera “eye” (distance + angle) once
        camera=dict(
          eye=dict(x=1.2, y=1.2, z=0.8)
        )
      )
    )
    
    fig = rotate_figure(fig, zoom_factor=zoom_factor*0.95, z_rotation=z_rotation, elev_rotation=elevation)
    
    grid_dict = dict(
          tickmode="array",     # custom list
          tickvals=[-2, -1, 0, 1, 2],
          ticktext=["-2","-1","0","1","2"],
          showgrid=True)
    
    fig.update_layout(
          scene=dict(
            xaxis=grid_dict,
            yaxis=grid_dict,
            zaxis=grid_dict
          ))
    fig.update_layout(
          coloraxis_showscale=False,  
          showlegend=False# hide
        )

    fig.write_image(os.path.join(temp_frame_path, f"hotfish_pca_temp{i:02}.png"), scale=2)
    fig.write_html(os.path.join(temp_frame_path, f"hotfish_pca_temp{i:02}.html"))

fig.show()

In [ ]:
angle_vec = np.linspace(0, 360, 50)

frame_path = os.path.join(fig_path, "hf_pca_rot_frames", "")
os.makedirs(frame_path, exist_ok=True)

# initialize figure
fig = px.scatter_3d(hf_pca_df, x="PCA_00_bio", y="PCA_01_bio", z="PCA_02_bio", 
                        color="temperature", symbol="timepoint", opacity=opacity,
                        color_continuous_scale="RdBu_r", range_color=[19, 38], 
                        hover_data={"timepoint", "snip_id"})

fig.layout.scene.xaxis.range = xrange
fig.layout.scene.yaxis.range = yrange
fig.layout.scene.zaxis.range = zrange

pca_axis_labels=["morph PC 1", "morph PC 2", "morph PC 3"]
fig = format_3d_plotly(fig, axis_labels=pca_axis_labels, theme="dark", marker_size=marker_size)

if t == 0:
    fig.add_trace(dummy_trace)
elif t == 1:
    fig.update_traces(marker=dict(line=dict(color="black")))

# rotate
for a, angle in enumerate(tqdm(angle_vec)):
    
    
    fig = rotate_figure(fig, zoom_factor=zoom_factor, z_rotation=z_rotation + angle, elev_rotation=elevation)
    

    fig.write_image(os.path.join(frame_path, f"hotfish_pca_rot{a:02}.png"), scale=2)
    # fig.write_html(os.path.join(fig_path, f"hotfish_pca_rot{a:02}.html"))
    
fig.show()